In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import pickle as pickle
import nltk
nltk.download('stopwords')


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from gensim.models.word2vec import Word2Vec
from sklearn import preprocessing,metrics 
pd.set_option("display.max_columns", 100)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model,svm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import average_precision_score, classification_report,confusion_matrix
from scipy.stats import skew
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk import word_tokenize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
classes = ['target', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack']
data = pd.read_csv('/content/drive/MyDrive/Untitled folder/train.csv').fillna(' ')
data['comment_text'].fillna("missing", inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data["comment_text"],data[classes],test_size=0.25,random_state=55)
data_text = pd.concat([X_train,X_test])

In [ ]:
data

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:41.987077+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:42.870083+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:45.222647+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2015-09-29 10:50:47.601894+00,2,,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0,0,0,0,0,0,0,0,0,0,0.25,0,0,0,0,0,0,0,0,0,0,0,0,0,2015-09-29 10:50:48.488476+00,2,,2006,rejected,0,0,0,1,0,0.0,4,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804869,6333967,0.000000,"Maybe the tax on ""things"" would be collected w...",0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 00:55:39.265997+00,54,6.33238e+06,399385,approved,0,0,0,0,0,0.0,0,4
1804870,6333969,0.000000,What do you call people who STILL think the di...,0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 00:55:56.634498+00,54,6.33387e+06,399528,approved,0,0,0,0,0,0.0,0,4
1804871,6333982,0.000000,"thank you ,,,right or wrong,,, i am following ...",0.000000,0.000000,0.000000,0.000000,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 00:59:02.840713+00,54,,399457,approved,0,0,0,0,0,0.0,0,4
1804872,6334009,0.621212,Anyone who is quoted as having the following e...,0.030303,0.030303,0.045455,0.621212,0.0,,,,,,,,,,,,,,,,,,,,,,,,,2017-11-11 01:01:10.153692+00,13,6.33383e+06,399519,approved,0,0,0,0,0,0.0,0,66


In [ ]:
#glove embedding
GLOVE_6B_50D_PATH = "/content/drive/MyDrive/Untitled folder/glove.6B.50d.txt"
glove = {}
glove_path = open(GLOVE_6B_50D_PATH)
for line in tqdm(glove_path):
    line_arr = line.split()
    word = line_arr[0]
    glove[word] = np.asarray(line_arr[1:], dtype='float32')
glove_path.close()

400000it [00:06, 59361.41it/s]


In [ ]:
#normalization 
nltk.download('punkt')
porter = PorterStemmer()
sw = set (stopwords.words('english'))
sw.update(['.', "'", '(', ')', '[', ']', '{', '}',',', '"', ':', ';'])
sentence= data_text
def vectorize(s):
    words = str (sentence).lower()
    tokenized = word_tokenize(words)
    updated_tokens=[]
    final_tokens = []
    for word in tokenized:
        if not word in sw and word.isalpha(): 
            updated_tokens.append(word)
    for token in updated_tokens:
        try:
            final_tokens.append(glove[token])
        except:
            continue  
    final_tokens = np.array(final_tokens)
    total =  final_tokens.sum(axis=0)
    data_type = type (total)
    if data_type != np.ndarray: return np.zeros(60)
    denom = np.sqrt((total ** 2).sum())
    return total / denom

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sw

{'"',
 "'",
 '(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '[',
 ']',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',

In [ ]:
X_train

88269      Dan, go back to your two bit rabble rousing. W...
1411672    "The Church is "always in need of reform". Can...
25238      Still peddling your idiotic comment, so-called...
1629159    other countries may pick the aspect of commerc...
1759496    So if the deficit increases 2.4 trillion over ...
                                 ...                        
1439677    I absolutely can not believe you would slam th...
1702429    So no it is not fair to small business owners ...
1131464    Thanks for saving me the typing, Heartland. Th...
1053338    2 tinfoil jokes in 1 section...how original. T...
1438157    The 2 biggest points I get from this article a...
Name: comment_text, Length: 1353655, dtype: object

In [ ]:
X_test

1765204    I just repost and keep reposting.  I will not ...
1088425    Mr. Bronskill might wish to "google" ... and t...
1093931    "real professional degrees" might include Medi...
509419     Well if you understood the 3 names portion of ...
1796202                                   I think I'll pass.
                                 ...                        
467714     Really they blamed RT for swinging the electio...
1387370    The Democrats are a bumbling, infighting miasm...
1490386    The NDP as usual promising the World to their ...
1592140    Even so, it strikes me as hypocritical that th...
1510863    Only in your *magical*; unreal world that is v...
Name: comment_text, Length: 451219, dtype: object

In [ ]:
xg_train = [vectorize(sentence) for sentence in tqdm(X_train)]
xg_test = [vectorize(sentence) for sentence in tqdm(X_test)]


Streaming output truncated to the last 5000 lines.

  0%|          | 384/1353655 [00:00<54:11, 416.20it/s]

  0%|          | 429/1353655 [00:01<53:11, 423.98it/s]

  0%|          | 474/1353655 [00:01<52:17, 431.26it/s]

  0%|          | 519/1353655 [00:01<51:52, 434.80it/s]

  0%|          | 563/1353655 [00:01<51:53, 434.64it/s]

  0%|          | 607/1353655 [00:01<53:02, 425.11it/s]

  0%|          | 650/1353655 [00:01<54:03, 417.16it/s]

  0%|          | 693/1353655 [00:01<53:49, 418.96it/s]

  0%|          | 735/1353655 [00:01<53:53, 418.44it/s]

  0%|          | 779/1353655 [00:01<53:19, 422.79it/s]

  0%|          | 822/1353655 [00:01<53:28, 421.68it/s]

  0%|          | 865/1353655 [00:02<53:33, 421.00it/s]

  0%|          | 908/1353655 [00:02<53:51, 418.65it/s]

  0%|          | 950/1353655 [00:02<54:24, 414.40it/s]

  0%|          | 992/1353655 [00:02<54:50, 411.09it/s]

  0%|          | 1034/1353655 [00:02<55:08, 408.78it/s]

  0%|          | 1078/1353655 [00:02<54:00, 417.35i

KeyboardInterrupt: ignored

In [ ]:
xg_train

In [ ]:
#logistic regression
logreg = OneVsRestClassifier(LogisticRegression())
logreg.fit(xg_train, y_train)
y_pred = logreg.predict_proba(xg_test)
accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(" Test Accuracy: %.2f%%" % (accuracy * 100.0))